In [0]:
WITH onlybaseentities as (
  SELECT
    child
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmparentchildhierarchy`
  WHERE
    `IsBaseRecord` = 1
    AND (
      `adjacencypath` LIKE
        'Entity|OEP_Total Entity|Total_Legal|Vertical_US_Newco_Inc_and_Subs|tk_Elevator_USA_Holding_CON|tk_Elevator_Americas_Corp_and_Subs|tk_Elevator_Corp_and_Subs|tk_Elevator_Corporate|901745_01_SAP|tk_Elevator_Corp_USField|%'
      OR `adjacencypath` LIKE
        'Entity|OEP_Total Entity|Total_Legal|thyssenkrupp_Canada_Holding_and_Subs|tk_Northern_Elevator_Corp_and_Subs|tk_Elevator_Canada_Ltd|%'
    )
),
branchnames as (
  SELECT
    backendname,
    alias
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmdimensionmetadata`
),
voluntaryturnover as (
  SELECT
    `Calendar Date`,
    branchid,
    branchname,
    branchcountry,
    sum(ftevoluntaryturnover) as turnover_all,
        sum(
      CASE
        WHEN
          costtype IN ('COS DIRECT')
        THEN
          coalesce(FTEVoluntaryTurnover, 0)
        ELSE 0
      END
    ) as turnover_direct,
    sum(
      CASE
        WHEN
          costtype IN ('COS DIRECT', 'SELLING')
        THEN
          coalesce(FTEVoluntaryTurnover, 0)
        ELSE 0
      END
    ) as turnover_direct_sales,
    sum(
      CASE
        WHEN
          costtype IN ('COS DIRECT', 'COS INDIRECT', 'SELLING')
        THEN
          coalesce(FTEVoluntaryTurnover, 0)
        ELSE 0
      END
    ) as turnover_direct_indirect_sales,
    sum(
      CASE
        WHEN
          costtype IN ('COS DIRECT', 'COS INDIRECT', 'SELLING', 'G&A')
        THEN
          coalesce(FTEVoluntaryTurnover, 0)
        ELSE 0
      END
    ) as turnover_direct_indirect_sales_ga
  FROM
    (
      SELECT
        TRUNC(TO_DATE(CAST(dateint AS STRING), 'yyyyMMdd'), 'MONTH') as `Calendar Date`,
        vt.epmbranch as branchid,
        branch.alias as branchname,
        vt.Country as branchcountry,
        costtype,
        ftevoluntaryturnover
      FROM
        `NA-CDP-BU-HR-DEV`.curated.voluntaryturnover as vt
          LEFT JOIN branchnames as branch
            on branch.backendname = vt.EPMBranch
    )
  GROUP BY
    `Calendar Date`,
    branchid,
    branchname,
    branchcountry
)
--SELECT * FROM voluntaryturnover where Year(`Calendar Date`) = 2025
SELECT
  *
FROM
  voluntaryturnover
where
  Year(`Calendar Date`) = 2025 and branchname = 'Houston LM'